In [16]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pylab
pylab.rcParams['figure.figsize'] = (8.0, 6.0)
from pycocotools.coco import COCO
import pickle

In [2]:
from config import ms_coco_dir
fc6_train = np.load(ms_coco_dir + '/descriptors/fc6/train.npy', encoding='latin1').item()
fc6_val = np.load(ms_coco_dir + '/descriptors/fc6/val.npy', encoding='latin1').item()
fc6_test = np.load(ms_coco_dir + '/descriptors/fc6/test.npy', encoding='latin1').item()

In [20]:
xc_val = pickle.load(open(ms_coco_dir + '/descriptors/xception/xception-val2017-descriptors.pic', 'rb'))

In [3]:
fc6 = dict(fc6_train)
fc6.update(fc6_val)
fc6.update(fc6_test)

In [4]:
del fc6_train
del fc6_val
del fc6_test

In [5]:
descriptors = fc6

In [6]:
normalized_descriptors = {k:v/np.linalg.norm(v) for k,v in descriptors.items()}

In [7]:
annFileTrain = '{}/annotations/captions_train2017.json'.format(ms_coco_dir)
annFileVal = '{}/annotations/captions_val2017.json'.format(ms_coco_dir)
cocoTrain = COCO(annFileTrain)
cocoVal = COCO(annFileVal)

loading annotations into memory...
Done (t=2.86s)
creating index...
index created!
loading annotations into memory...
Done (t=0.16s)
creating index...
index created!


In [8]:
trainIds = [i for i in cocoTrain.getImgIds() if i in descriptors]
valIds   = [i for i in cocoVal.getImgIds() if i in descriptors]

In [9]:
# trainIds_r = {v:i for i,v in enumerate(trainIds)}
# valIds_r = {v:i for i,v in enumerate(valIds)}

In [10]:
# tt = [np.dot(descriptors[x], descriptors[x]) for x in trainIds]
# vv = [np.dot(descriptors[x], descriptors[x]) for x in valIds]

In [11]:
import time


def eucleidean_dist(x, y):  # not really an eucleidean distance
    z = x - y
    return np.dot(z, z)    

def cosine_dist(x, y):
    return 1 - np.dot(x,y)**2 / np.dot(x,x) / np.dot(y,y)

def cosine_dist2(x, y, xx, yy):
    return 1 - np.dot(x,y)**2 / xx / yy

def cosine_dist3(x, y):  # supposes x and y were normalized to norm = 1
    return 1 - np.dot(x,y)

def knn(v_id):
    min_d = np.infty
    min_id = None
    for t_id in trainIds:
#         d = cosine_dist2(descriptors[t_id], descriptors[v_id], tt[trainIds_r[t_id]], vv[valIds_r[v_id]])
        d = cosine_dist3(normalized_descriptors[t_id], normalized_descriptors[v_id])
#         d = eucleidean_dist(descriptors[t_id], descriptors[v_id])
        if d < min_d:
            min_d = d
            min_id = t_id
    return min_id    

def predict_val():
    predictions = {}
    t_start = time.clock()
    for i, v_id in enumerate(valIds):
        print('{}/{}'.format(i+1, len(valIds)))
        t_id = knn(v_id) ######
        if t_id is None:
            continue
        annIds = cocoTrain.getAnnIds(imgIds=[t_id])
        anns = cocoTrain.loadAnns(annIds)[0] # take the first caption
        predictions[v_id] = anns['caption']
        
        t_now = time.clock()
        t_elapsed = t_now - t_start
        str_elapsed = time.strftime('%Hh %Mm %Ss', time.gmtime(t_elapsed))
        t_eta = t_elapsed / (i+1) * (len(valIds)-i-1)
        str_eta = time.strftime('%Hh %Mm %Ss', time.gmtime(t_eta))
        print('elapsed:', str_elapsed, '. ETA:', str_eta)
    return predictions

In [12]:
# %load_ext line_profiler
# %lprun -f knn predict_val()
predictions = predict_val()

1/4942
elapsed: 00h 00m 01s . ETA: 01h 43m 03s
2/4942
elapsed: 00h 00m 02s . ETA: 01h 41m 55s
3/4942
elapsed: 00h 00m 03s . ETA: 01h 41m 11s
4/4942
elapsed: 00h 00m 04s . ETA: 01h 40m 58s
5/4942
elapsed: 00h 00m 06s . ETA: 01h 40m 58s
6/4942
elapsed: 00h 00m 07s . ETA: 01h 41m 23s
7/4942
elapsed: 00h 00m 08s . ETA: 01h 42m 32s
8/4942
elapsed: 00h 00m 09s . ETA: 01h 42m 30s
9/4942
elapsed: 00h 00m 11s . ETA: 01h 42m 16s
10/4942
elapsed: 00h 00m 12s . ETA: 01h 42m 07s
11/4942
elapsed: 00h 00m 13s . ETA: 01h 42m 09s
12/4942
elapsed: 00h 00m 14s . ETA: 01h 42m 09s
13/4942
elapsed: 00h 00m 16s . ETA: 01h 42m 25s
14/4942
elapsed: 00h 00m 17s . ETA: 01h 42m 23s
15/4942
elapsed: 00h 00m 18s . ETA: 01h 42m 27s
16/4942
elapsed: 00h 00m 19s . ETA: 01h 42m 37s
17/4942
elapsed: 00h 00m 21s . ETA: 01h 42m 32s
18/4942
elapsed: 00h 00m 22s . ETA: 01h 42m 30s
19/4942
elapsed: 00h 00m 23s . ETA: 01h 42m 20s
20/4942
elapsed: 00h 00m 24s . ETA: 01h 42m 12s
21/4942
elapsed: 00h 00m 26s . ETA: 01h 42m 07s
2

elapsed: 00h 03m 35s . ETA: 01h 40m 03s
172/4942
elapsed: 00h 03m 36s . ETA: 01h 40m 06s
173/4942
elapsed: 00h 03m 37s . ETA: 01h 40m 08s
174/4942
elapsed: 00h 03m 39s . ETA: 01h 40m 08s
175/4942
elapsed: 00h 03m 40s . ETA: 01h 40m 12s
176/4942
elapsed: 00h 03m 42s . ETA: 01h 40m 16s
177/4942
elapsed: 00h 03m 43s . ETA: 01h 40m 17s
178/4942
elapsed: 00h 03m 44s . ETA: 01h 40m 17s
179/4942
elapsed: 00h 03m 46s . ETA: 01h 40m 20s
180/4942
elapsed: 00h 03m 47s . ETA: 01h 40m 24s
181/4942
elapsed: 00h 03m 49s . ETA: 01h 40m 24s
182/4942
elapsed: 00h 03m 50s . ETA: 01h 40m 26s
183/4942
elapsed: 00h 03m 51s . ETA: 01h 40m 27s
184/4942
elapsed: 00h 03m 53s . ETA: 01h 40m 30s
185/4942
elapsed: 00h 03m 54s . ETA: 01h 40m 29s
186/4942
elapsed: 00h 03m 55s . ETA: 01h 40m 28s
187/4942
elapsed: 00h 03m 57s . ETA: 01h 40m 33s
188/4942
elapsed: 00h 03m 58s . ETA: 01h 40m 33s
189/4942
elapsed: 00h 03m 59s . ETA: 01h 40m 31s
190/4942
elapsed: 00h 04m 01s . ETA: 01h 40m 30s
191/4942
elapsed: 00h 04m 02s

elapsed: 00h 07m 07s . ETA: 01h 36m 47s
340/4942
elapsed: 00h 07m 08s . ETA: 01h 36m 45s
341/4942
elapsed: 00h 07m 10s . ETA: 01h 36m 43s
342/4942
elapsed: 00h 07m 11s . ETA: 01h 36m 41s
343/4942
elapsed: 00h 07m 12s . ETA: 01h 36m 40s
344/4942
elapsed: 00h 07m 13s . ETA: 01h 36m 38s
345/4942
elapsed: 00h 07m 15s . ETA: 01h 36m 36s
346/4942
elapsed: 00h 07m 16s . ETA: 01h 36m 35s
347/4942
elapsed: 00h 07m 17s . ETA: 01h 36m 33s
348/4942
elapsed: 00h 07m 18s . ETA: 01h 36m 31s
349/4942
elapsed: 00h 07m 19s . ETA: 01h 36m 30s
350/4942
elapsed: 00h 07m 21s . ETA: 01h 36m 28s
351/4942
elapsed: 00h 07m 22s . ETA: 01h 36m 26s
352/4942
elapsed: 00h 07m 23s . ETA: 01h 36m 24s
353/4942
elapsed: 00h 07m 24s . ETA: 01h 36m 23s
354/4942
elapsed: 00h 07m 26s . ETA: 01h 36m 21s
355/4942
elapsed: 00h 07m 27s . ETA: 01h 36m 19s
356/4942
elapsed: 00h 07m 28s . ETA: 01h 36m 17s
357/4942
elapsed: 00h 07m 29s . ETA: 01h 36m 16s
358/4942
elapsed: 00h 07m 30s . ETA: 01h 36m 14s
359/4942
elapsed: 00h 07m 32s

elapsed: 00h 10m 34s . ETA: 01h 32m 28s
508/4942
elapsed: 00h 10m 35s . ETA: 01h 32m 27s
509/4942
elapsed: 00h 10m 36s . ETA: 01h 32m 25s
510/4942
elapsed: 00h 10m 37s . ETA: 01h 32m 24s
511/4942
elapsed: 00h 10m 39s . ETA: 01h 32m 22s
512/4942
elapsed: 00h 10m 40s . ETA: 01h 32m 21s
513/4942
elapsed: 00h 10m 41s . ETA: 01h 32m 19s
514/4942
elapsed: 00h 10m 42s . ETA: 01h 32m 18s
515/4942
elapsed: 00h 10m 44s . ETA: 01h 32m 17s
516/4942
elapsed: 00h 10m 45s . ETA: 01h 32m 15s
517/4942
elapsed: 00h 10m 46s . ETA: 01h 32m 14s
518/4942
elapsed: 00h 10m 47s . ETA: 01h 32m 12s
519/4942
elapsed: 00h 10m 49s . ETA: 01h 32m 11s
520/4942
elapsed: 00h 10m 50s . ETA: 01h 32m 09s
521/4942
elapsed: 00h 10m 51s . ETA: 01h 32m 08s
522/4942
elapsed: 00h 10m 52s . ETA: 01h 32m 06s
523/4942
elapsed: 00h 10m 53s . ETA: 01h 32m 05s
524/4942
elapsed: 00h 10m 55s . ETA: 01h 32m 04s
525/4942
elapsed: 00h 10m 56s . ETA: 01h 32m 02s
526/4942
elapsed: 00h 10m 57s . ETA: 01h 32m 01s
527/4942
elapsed: 00h 10m 58s

elapsed: 00h 14m 00s . ETA: 01h 28m 31s
676/4942
elapsed: 00h 14m 01s . ETA: 01h 28m 30s
677/4942
elapsed: 00h 14m 02s . ETA: 01h 28m 29s
678/4942
elapsed: 00h 14m 03s . ETA: 01h 28m 27s
679/4942
elapsed: 00h 14m 05s . ETA: 01h 28m 26s
680/4942
elapsed: 00h 14m 06s . ETA: 01h 28m 25s
681/4942
elapsed: 00h 14m 07s . ETA: 01h 28m 23s
682/4942
elapsed: 00h 14m 08s . ETA: 01h 28m 22s
683/4942
elapsed: 00h 14m 10s . ETA: 01h 28m 21s
684/4942
elapsed: 00h 14m 11s . ETA: 01h 28m 19s
685/4942
elapsed: 00h 14m 12s . ETA: 01h 28m 18s
686/4942
elapsed: 00h 14m 13s . ETA: 01h 28m 17s
687/4942
elapsed: 00h 14m 15s . ETA: 01h 28m 15s
688/4942
elapsed: 00h 14m 16s . ETA: 01h 28m 14s
689/4942
elapsed: 00h 14m 17s . ETA: 01h 28m 13s
690/4942
elapsed: 00h 14m 18s . ETA: 01h 28m 11s
691/4942
elapsed: 00h 14m 19s . ETA: 01h 28m 10s
692/4942
elapsed: 00h 14m 21s . ETA: 01h 28m 09s
693/4942
elapsed: 00h 14m 22s . ETA: 01h 28m 07s
694/4942
elapsed: 00h 14m 23s . ETA: 01h 28m 06s
695/4942
elapsed: 00h 14m 24s

...

elapsed: 01h 24m 22s . ETA: 00h 16m 41s
4127/4942
elapsed: 01h 24m 24s . ETA: 00h 16m 40s
4128/4942
elapsed: 01h 24m 25s . ETA: 00h 16m 38s
4129/4942
elapsed: 01h 24m 26s . ETA: 00h 16m 37s
4130/4942
elapsed: 01h 24m 27s . ETA: 00h 16m 36s
4131/4942
elapsed: 01h 24m 28s . ETA: 00h 16m 35s
4132/4942
elapsed: 01h 24m 30s . ETA: 00h 16m 33s
4133/4942
elapsed: 01h 24m 31s . ETA: 00h 16m 32s
4134/4942
elapsed: 01h 24m 32s . ETA: 00h 16m 31s
4135/4942
elapsed: 01h 24m 33s . ETA: 00h 16m 30s
4136/4942
elapsed: 01h 24m 34s . ETA: 00h 16m 28s
4137/4942
elapsed: 01h 24m 36s . ETA: 00h 16m 27s
4138/4942
elapsed: 01h 24m 37s . ETA: 00h 16m 26s
4139/4942
elapsed: 01h 24m 38s . ETA: 00h 16m 25s
4140/4942
elapsed: 01h 24m 39s . ETA: 00h 16m 24s
4141/4942
elapsed: 01h 24m 41s . ETA: 00h 16m 22s
4142/4942
elapsed: 01h 24m 42s . ETA: 00h 16m 21s
4143/4942
elapsed: 01h 24m 43s . ETA: 00h 16m 20s
4144/4942
elapsed: 01h 24m 44s . ETA: 00h 16m 19s
4145/4942
elapsed: 01h 24m 45s . ETA: 00h 16m 17s
4146/4942


elapsed: 01h 27m 45s . ETA: 00h 13m 20s
4291/4942
elapsed: 01h 27m 46s . ETA: 00h 13m 19s
4292/4942
elapsed: 01h 27m 48s . ETA: 00h 13m 17s
4293/4942
elapsed: 01h 27m 49s . ETA: 00h 13m 16s
4294/4942
elapsed: 01h 27m 50s . ETA: 00h 13m 15s
4295/4942
elapsed: 01h 27m 51s . ETA: 00h 13m 14s
4296/4942
elapsed: 01h 27m 52s . ETA: 00h 13m 12s
4297/4942
elapsed: 01h 27m 54s . ETA: 00h 13m 11s
4298/4942
elapsed: 01h 27m 55s . ETA: 00h 13m 10s
4299/4942
elapsed: 01h 27m 56s . ETA: 00h 13m 09s
4300/4942
elapsed: 01h 27m 57s . ETA: 00h 13m 07s
4301/4942
elapsed: 01h 27m 59s . ETA: 00h 13m 06s
4302/4942
elapsed: 01h 28m 00s . ETA: 00h 13m 05s
4303/4942
elapsed: 01h 28m 01s . ETA: 00h 13m 04s
4304/4942
elapsed: 01h 28m 02s . ETA: 00h 13m 03s
4305/4942
elapsed: 01h 28m 03s . ETA: 00h 13m 01s
4306/4942
elapsed: 01h 28m 05s . ETA: 00h 13m 00s
4307/4942
elapsed: 01h 28m 06s . ETA: 00h 12m 59s
4308/4942
elapsed: 01h 28m 07s . ETA: 00h 12m 58s
4309/4942
elapsed: 01h 28m 08s . ETA: 00h 12m 56s
4310/4942


elapsed: 01h 31m 06s . ETA: 00h 09m 58s
4455/4942
elapsed: 01h 31m 07s . ETA: 00h 09m 57s
4456/4942
elapsed: 01h 31m 08s . ETA: 00h 09m 56s
4457/4942
elapsed: 01h 31m 09s . ETA: 00h 09m 55s
4458/4942
elapsed: 01h 31m 11s . ETA: 00h 09m 53s
4459/4942
elapsed: 01h 31m 12s . ETA: 00h 09m 52s
4460/4942
elapsed: 01h 31m 13s . ETA: 00h 09m 51s
4461/4942
elapsed: 01h 31m 14s . ETA: 00h 09m 50s
4462/4942
elapsed: 01h 31m 15s . ETA: 00h 09m 49s
4463/4942
elapsed: 01h 31m 17s . ETA: 00h 09m 47s
4464/4942
elapsed: 01h 31m 18s . ETA: 00h 09m 46s
4465/4942
elapsed: 01h 31m 19s . ETA: 00h 09m 45s
4466/4942
elapsed: 01h 31m 20s . ETA: 00h 09m 44s
4467/4942
elapsed: 01h 31m 22s . ETA: 00h 09m 42s
4468/4942
elapsed: 01h 31m 23s . ETA: 00h 09m 41s
4469/4942
elapsed: 01h 31m 24s . ETA: 00h 09m 40s
4470/4942
elapsed: 01h 31m 25s . ETA: 00h 09m 39s
4471/4942
elapsed: 01h 31m 27s . ETA: 00h 09m 38s
4472/4942
elapsed: 01h 31m 28s . ETA: 00h 09m 36s
4473/4942
elapsed: 01h 31m 29s . ETA: 00h 09m 35s
4474/4942


elapsed: 01h 34m 26s . ETA: 00h 06m 37s
4619/4942
elapsed: 01h 34m 28s . ETA: 00h 06m 36s
4620/4942
elapsed: 01h 34m 29s . ETA: 00h 06m 35s
4621/4942
elapsed: 01h 34m 30s . ETA: 00h 06m 33s
4622/4942
elapsed: 01h 34m 31s . ETA: 00h 06m 32s
4623/4942
elapsed: 01h 34m 32s . ETA: 00h 06m 31s
4624/4942
elapsed: 01h 34m 34s . ETA: 00h 06m 30s
4625/4942
elapsed: 01h 34m 35s . ETA: 00h 06m 28s
4626/4942
elapsed: 01h 34m 36s . ETA: 00h 06m 27s
4627/4942
elapsed: 01h 34m 37s . ETA: 00h 06m 26s
4628/4942
elapsed: 01h 34m 39s . ETA: 00h 06m 25s
4629/4942
elapsed: 01h 34m 40s . ETA: 00h 06m 24s
4630/4942
elapsed: 01h 34m 41s . ETA: 00h 06m 22s
4631/4942
elapsed: 01h 34m 42s . ETA: 00h 06m 21s
4632/4942
elapsed: 01h 34m 43s . ETA: 00h 06m 20s
4633/4942
elapsed: 01h 34m 45s . ETA: 00h 06m 19s
4634/4942
elapsed: 01h 34m 46s . ETA: 00h 06m 17s
4635/4942
elapsed: 01h 34m 47s . ETA: 00h 06m 16s
4636/4942
elapsed: 01h 34m 48s . ETA: 00h 06m 15s
4637/4942
elapsed: 01h 34m 50s . ETA: 00h 06m 14s
4638/4942


elapsed: 01h 37m 47s . ETA: 00h 03m 16s
4783/4942
elapsed: 01h 37m 48s . ETA: 00h 03m 15s
4784/4942
elapsed: 01h 37m 49s . ETA: 00h 03m 13s
4785/4942
elapsed: 01h 37m 50s . ETA: 00h 03m 12s
4786/4942
elapsed: 01h 37m 51s . ETA: 00h 03m 11s
4787/4942
elapsed: 01h 37m 53s . ETA: 00h 03m 10s
4788/4942
elapsed: 01h 37m 54s . ETA: 00h 03m 08s
4789/4942
elapsed: 01h 37m 55s . ETA: 00h 03m 07s
4790/4942
elapsed: 01h 37m 56s . ETA: 00h 03m 06s
4791/4942
elapsed: 01h 37m 58s . ETA: 00h 03m 05s
4792/4942
elapsed: 01h 37m 59s . ETA: 00h 03m 04s
4793/4942
elapsed: 01h 38m 00s . ETA: 00h 03m 02s
4794/4942
elapsed: 01h 38m 01s . ETA: 00h 03m 01s
4795/4942
elapsed: 01h 38m 02s . ETA: 00h 03m 00s
4796/4942
elapsed: 01h 38m 04s . ETA: 00h 02m 59s
4797/4942
elapsed: 01h 38m 05s . ETA: 00h 02m 57s
4798/4942
elapsed: 01h 38m 06s . ETA: 00h 02m 56s
4799/4942
elapsed: 01h 38m 07s . ETA: 00h 02m 55s
4800/4942
elapsed: 01h 38m 08s . ETA: 00h 02m 54s
4801/4942
elapsed: 01h 38m 10s . ETA: 00h 02m 52s
4802/4942


In [13]:
import pickle
with open('predictions-knn-cosine.pic', 'wb') as f:
    pickle.dump(predictions, f)

In [14]:
import pickle
with open('predictions-knn-cosine.pic', 'rb') as f:
    predictions = pickle.load(f)

In [15]:
import json
predictions_list = [{'image_id':iid, 'caption':caption} for iid, caption in predictions.items()]
with open('predictions-knn-cosine.json', 'w') as f:
    json.dump(predictions_list, f)